In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
batch_size = 100
learning_rate = 0.001
num_epoch = 10

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

In [10]:
x = datasets.MNIST(root = "./data",train = True , transform = transform, download = True)
y = datasets.MNIST(root = "./data",train = False, transform=transform, download = True)

In [11]:
x_loader = DataLoader(dataset = x , batch_size = batch_size, shuffle = True)
y_loader = DataLoader(dataset = y, batch_size= batch_size, shuffle = False)

In [39]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size = 3,stride= 1,padding= 1)
    self.conv2 = nn.Conv2d(32,64,kernel_size = 3,stride = 1,padding= 1)
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.fc1 = nn.Linear(64*7*7,128)
    self.fc2 = nn.Linear(128,10)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.conv1(x)
    x = self.pool(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.pool(x)
    x = self.relu(x)
    x = x.view(x.size(0),-1)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [40]:
model = CNN().to(device)

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [42]:
for epoch in range(num_epoch):
  model.train()
  running_loss = 0.0
  for images, labels in x_loader:
    images,labels = images.to(device),labels.to(device)

    output = model(images)
    loss = criterion(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'Epoch [{epoch + 1}/{num_epoch}], Loss: {running_loss / len(x_loader):.4f}')

Epoch [1/10], Loss: 0.1948
Epoch [2/10], Loss: 0.0501
Epoch [3/10], Loss: 0.0350
Epoch [4/10], Loss: 0.0264
Epoch [5/10], Loss: 0.0219
Epoch [6/10], Loss: 0.0161
Epoch [7/10], Loss: 0.0124
Epoch [8/10], Loss: 0.0112
Epoch [9/10], Loss: 0.0098
Epoch [10/10], Loss: 0.0081


In [46]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images,labels in y_loader:
    images,labels = images.to(device),labels.to(device)
    output = model(images)
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100*correct/total
print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 99.14%


In [47]:
torch.save(model.state_dict(),"mnist-cnn.pth")
print("Model saved as mnist_cnn.pth")

Model saved as mnist_cnn.pth
